In [ ]:
# Instalação das bibliotecas necessárias no ambiente do Google Colab
# reportlab: para a geração do arquivo PDF
# pandas: para manipulação e análise de dados tabulares
# openpyxl: para ler arquivos Excel (.xlsx)
# requests: para fazer requisições HTTP a APIs externas
!pip install reportlab pandas openpyxl requests

# 1. --- Importação das bibliotecas ---
import pandas as pd
import io
# `google.colab.files` é usado para interagir com o sistema de arquivos do Colab,
# permitindo o upload e download de arquivos
from google.colab import files
# Módulos do `reportlab` para criar o documento PDF
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
from reportlab.lib.colors import green, red, orange, black
# Módulos padrão do Python
import re
import numpy as np
import requests
import time

# 2. --- Funções auxiliares ---

def formatar_moeda(valor):
    """
    Formata um valor numérico (float) para uma string com o formato de moeda brasileira (R$).
    Ele lida com valores NaN (not a number) e converte o ponto decimal para vírgula.
    """
    if pd.isna(valor):
        return "R$ 0,00"
    return f"R$ {valor:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

# Dicionário para mapear números de mês para seus nomes em português
meses_portugues = {
    1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril', 5: 'Maio', 6: 'Junho',
    7: 'Julho', 8: 'Agosto', 9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'
}

# Constante que define o limite de faturamento anual para o Simples Nacional
LIMITE_SIMPLES = 4800000

# ATENÇÃO: COLOQUE SUA CHAVE DE API AQUI!
# Esta chave é essencial para a consulta do status de CNPJs
API_TOKEN = "21658|9svRpIVCYW0qmFiQ0SudnecBoHUuowGY"

def calcular_simples_nacional_anexo3(faturamento_anual):
    """
    Calcula o valor do imposto estimado com base no faturamento anual,
    usando a tabela do Anexo III do Simples Nacional.
    A função percorre as faixas de faturamento e aplica a alíquota e a parcela
    a deduzir correspondentes.
    """
    tabelas = [
        (180000, 0.06, 0),
        (360000, 0.112, 9360),
        (720000, 0.135, 17640),
        (1800000, 0.175, 35640),
        (3600000, 0.21, 125640),
        (4800000, 0.33, 648000)
    ]
    for limite, aliquota, parcela_deduzir in tabelas:
        if faturamento_anual <= limite:
            return (faturamento_anual * aliquota) - parcela_deduzir
    return faturamento_anual * 0.33

def verificar_mudanca_faixa(faturamento_anual):
    """
    Verifica se o faturamento anual está próximo de um limite de faixa do Simples Nacional.
    Retorna uma string formatada com cor, alertando o usuário se o faturamento
    estiver a menos de 10% de um novo limite, indicando um possível risco fiscal.
    """
    faixas = [180000, 360000, 720000, 1800000, 3600000, 4800000]
    for limite in faixas:
        if faturamento_anual < limite:
            restante = limite - faturamento_anual
            if restante <= limite * 0.1:
                return f'<font color="red">■ [ATENÇÃO]: PRÓXIMO DE MUDANÇA DE FAIXA. Restante para o próximo limite: {formatar_moeda(restante)}</font>'
            break
    return '<font color="green">■ [OK]: Faturamento dentro da faixa atual.</font>'

def verificar_status_cnpj(cnpj, max_retries=3):
    """
    Consulta a situação de um CNPJ em uma API externa com múltiplas tentativas.
    Limpa o CNPJ para conter apenas dígitos e faz uma requisição GET para a API.
    Lida com diferentes códigos de status HTTP e erros de requisição.
    """
    cnpj_limpo = ''.join(filter(str.isdigit, str(cnpj)))
    url = f"https://api.invertexto.com/v1/cnpj/{cnpj_limpo}?token={API_TOKEN}"
    for attempt in range(max_retries):
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                # Extrai o nome da situação legal do CNPJ
                status_legal = data.get("situacao", {}).get("nome") or "DESCONHECIDO"
                return status_legal
            elif response.status_code == 401:
                return "TOKEN INVÁLIDO"
            elif response.status_code == 404:
                return "NÃO ENCONTRADO"
            else:
                response.raise_for_status()
        except requests.exceptions.RequestException:
            # Em caso de erro, aguarda 5 segundos antes de tentar novamente
            if attempt < max_retries - 1:
                time.sleep(5)
                continue
            else:
                return "ERRO NA CONSULTA"

def gerar_pdf(story, nome_arquivo="relatorio_fiscal.pdf"):
    """
    Cria e salva o documento PDF a partir do conteúdo (`story`) gerado.
    """
    doc = SimpleDocTemplate(nome_arquivo, pagesize=letter)
    doc.build(story)

# 3. --- Upload de arquivos ---
print("Selecione os arquivos: notas_fiscais e extrato_bancario")
# Abre uma caixa de diálogo para o usuário fazer o upload dos arquivos
uploaded = files.upload()

# 4. --- Leitura e padronização dos DataFrames ---
notas_df = None
extrato_df = None

def encontrar_e_renomear_coluna(df, keywords):
    """
    Função auxiliar para encontrar e retornar o nome de uma coluna
    baseada em uma lista de palavras-chave, independentemente da capitalização
    e de acentuação (ex: 'ç' e 'ã').
    """
    for col in df.columns:
        col_limpa = col.lower().replace('ç', 'c').replace('ã', 'a')
        for keyword in keywords:
            if keyword in col_limpa:
                return col
    return None

# Loop que processa os arquivos enviados pelo usuário
for file_name, file_content in uploaded.items():
    try:
        # Verifica se o nome do arquivo contém 'notas' (para o arquivo de notas fiscais)
        if 'notas' in file_name.lower():
            if file_name.endswith('.csv'):
                # Decodifica o arquivo CSV e lê com o pandas
                content = file_content.decode('latin1')
                notas_df = pd.read_csv(io.StringIO(content), sep=';', dtype=str)
            elif file_name.endswith('.xlsx'):
                # Lê o arquivo Excel com o pandas
                notas_df = pd.read_excel(io.BytesIO(file_content), dtype=str)

            # Renomeia as colunas do DataFrame para um padrão fixo ('Data', 'CNPJCliente', 'Valor')
            notas_df.rename(columns={
                encontrar_e_renomear_coluna(notas_df, ['data', 'emissao']): 'Data',
                encontrar_e_renomear_coluna(notas_df, ['cnpj', 'emissor']): 'CNPJCliente',
                encontrar_e_renomear_coluna(notas_df, ['valor', 'total']): 'Valor'
            }, inplace=True)
            print(f"Colunas do DataFrame de Notas Fiscais após renomear: {notas_df.columns.tolist()}")

        # Verifica se o nome do arquivo contém 'extrato' (para o arquivo de extrato bancário)
        elif 'extrato' in file_name.lower():
            if file_name.endswith('.csv'):
                content = file_content.decode('latin1')
                extrato_df = pd.read_csv(io.StringIO(content), sep=';', dtype=str)
            elif file_name.endswith('.xlsx'):
                extrato_df = pd.read_excel(io.BytesIO(file_content), dtype=str)

            # Renomeia explicitamente a coluna "DescriÃ§Ã£o" para corrigir problemas de codificação
            extrato_df.rename(columns={'DescriÃ§Ã£o': 'Descricao'}, inplace=True, errors='ignore')

            # Renomeia as colunas do extrato para um padrão fixo ('Data', 'Valor')
            extrato_df.rename(columns={
                encontrar_e_renomear_coluna(extrato_df, ['data']): 'Data',
                encontrar_e_renomear_coluna(extrato_df, ['valor']): 'Valor'
            }, inplace=True)
            print(f"Colunas do DataFrame de Extrato Bancário após renomear: {extrato_df.columns.tolist()}")

    except Exception as e:
        print(f"Erro ao ler ou processar o arquivo '{file_name}': {e}")

# Verifica se ambos os arquivos foram carregados corretamente
if notas_df is None or extrato_df is None:
    print("Erro: Não foi possível carregar 'notas fiscais' e 'extrato bancário'. Verifique os nomes dos arquivos.")
    exit()

# 5. --- Pós-processamento e análise de dados ---
# Limpa os dados das colunas de 'Valor', removendo caracteres de moeda, pontos e vírgulas
notas_df['Valor'] = notas_df['Valor'].astype(str).str.replace(r'[R$\s\.]', '', regex=True).str.replace(',', '.')
extrato_df['Valor'] = extrato_df['Valor'].astype(str).str.replace(r'[R$\s\.]', '', regex=True).str.replace(',', '.')

# Converte as colunas 'Valor' para tipo numérico e remove as linhas com valores inválidos
notas_df['Valor'] = pd.to_numeric(notas_df['Valor'], errors='coerce')
extrato_df['Valor'] = pd.to_numeric(extrato_df['Valor'], errors='coerce')

notas_df.dropna(subset=['Valor'], inplace=True)
extrato_df.dropna(subset=['Valor'], inplace=True)

# Converte as colunas 'Data' para o formato de data e hora do pandas
notas_df['Data'] = pd.to_datetime(notas_df['Data'], format="%d/%m/%Y", errors='coerce')
extrato_df['Data'] = pd.to_datetime(extrato_df['Data'], format="%d/%m/%Y", errors='coerce')

# Remove linhas com datas inválidas após a conversão
notas_df.dropna(subset=['Data'], inplace=True)
extrato_df.dropna(subset=['Data'], inplace=True)

# 6. --- Geração do relatório em PDF (agora com formatação direta e por mês) ---
# Define os estilos de parágrafo a serem usados no PDF
styles = getSampleStyleSheet()
normal_style = styles['Normal']
heading1_style = styles['Heading1']
heading2_style = styles['Heading2']
heading3_style = styles['Heading3']

# A lista `story` conterá todos os elementos do PDF (parágrafos, espaçadores, etc.)
story = []

# Adiciona o título principal do relatório e um espaçador
story.append(Paragraph("Relatório de Análise Fiscal", heading1_style))
story.append(Spacer(1, 0.2*inch))

# Extrai o ano e o mês de cada DataFrame para agrupar os dados
notas_df['Ano'] = notas_df['Data'].dt.year
notas_df['Mes'] = notas_df['Data'].dt.month
extrato_df['Ano'] = extrato_df['Data'].dt.year
extrato_df['Mes'] = extrato_df['Data'].dt.month

# Cria uma lista de tuplas (ano, mês) únicos para iterar sobre todos os períodos de dados
datas_unicas = sorted(list(set([tuple(x) for x in notas_df[['Ano', 'Mes']].values.tolist() + extrato_df[['Ano', 'Mes']].values.tolist()])))

impostos_mensais = {}

is_first_page = True
for ano, mes in datas_unicas:
    # Insere uma quebra de página antes de cada novo mês, exceto no primeiro
    if not is_first_page:
        story.append(PageBreak())
    is_first_page = False

    # Adiciona o cabeçalho para o mês/ano atual
    story.append(Paragraph(f"Análise de Faturamento: {meses_portugues[mes]}/{int(ano)}", heading2_style))
    story.append(Spacer(1, 0.1*inch))

    # Filtra os dados de notas e extrato apenas para o mês e ano do loop
    notas_do_mes = notas_df[(notas_df['Ano'] == ano) & (notas_df['Mes'] == mes)].copy()
    extrato_do_mes = extrato_df[(extrato_df['Ano'] == ano) & (extrato_df['Mes'] == mes)].copy()

    # Calcula e adiciona ao PDF o faturamento e a estimativa de imposto do mês
    faturamento_mensal = notas_do_mes['Valor'].sum()
    story.append(Paragraph(f"Faturamento Total do Mês: <b>{formatar_moeda(faturamento_mensal)}</b>", normal_style))

    imposto_mensal = calcular_simples_nacional_anexo3(faturamento_mensal)
    impostos_mensais[(ano, mes)] = imposto_mensal
    story.append(Paragraph(f"Estimativa de Imposto (Simples Nacional): <b>{formatar_moeda(imposto_mensal)}</b>", normal_style))
    story.append(Spacer(1, 0.1*inch))

    # Lista todas as notas fiscais do mês
    story.append(Paragraph("Notas Fiscais do Mês:", heading3_style))
    if not notas_do_mes.empty:
        for _, row in notas_do_mes.iterrows():
            story.append(Paragraph(f"• Data: {row['Data'].strftime('%d/%m/%Y')}, CNPJ: {row['CNPJCliente']}, Valor: {formatar_moeda(row['Valor'])}", normal_style))
    else:
        story.append(Paragraph("Nenhuma nota fiscal emitida neste mês.", normal_style))
    story.append(Spacer(1, 0.1*inch))

    # Lista todas as movimentações do extrato bancário do mês
    story.append(Paragraph("Movimentações do Extrato Bancário:", heading3_style))
    if not extrato_do_mes.empty:
        for _, row in extrato_do_mes.iterrows():
            descricao = row.get('Descricao', 'Sem descrição')
            story.append(Paragraph(f"• Data: {row['Data'].strftime('%d/%m/%Y')}, Descrição: {descricao}, Valor: {formatar_moeda(row['Valor'])}", normal_style))
    else:
        story.append(Paragraph("Nenhuma movimentação no extrato neste mês.", normal_style))
    story.append(Spacer(1, 0.1*inch))

    # Alertas de Inconsistência do Mês
    story.append(Paragraph("Alertas de Inconsistência:", heading3_style))

    # Encontra notas fiscais que não tiveram um recebimento correspondente no extrato
    # A verificação é feita dentro de uma janela de 7 dias após a data da nota
    extrato_recebimentos = extrato_do_mes[extrato_do_mes['Valor'] > 0]
    notas_sem_corresp = []
    for _, nota in notas_do_mes.iterrows():
        janela = (extrato_recebimentos['Data'] >= nota['Data']) & (extrato_recebimentos['Data'] <= nota['Data'] + pd.Timedelta(days=7))
        # np.isclose é usado para comparar valores de ponto flutuante com tolerância
        if not extrato_recebimentos[janela & (np.isclose(extrato_recebimentos['Valor'], nota['Valor'], atol=0.01))].empty:
            continue
        notas_sem_corresp.append(nota)

    # Adiciona o alerta de risco fiscal ao PDF
    if notas_sem_corresp:
        story.append(Paragraph('<font color="red">■</font> [RISCO FISCAL]: Notas Fiscais emitidas sem recebimento correspondente:', normal_style))
        for nota in notas_sem_corresp:
            story.append(Paragraph(f"  - Nota de {formatar_moeda(nota['Valor'])} de {nota['Data'].strftime('%d/%m/%Y')} para o CNPJ {nota['CNPJCliente']}.", normal_style))
    else:
        story.append(Paragraph('<font color="green">■</font> [OK]: Todas as notas do mês têm correspondência no extrato.', normal_style))
    story.append(Spacer(1, 0.1*inch))

    # Alerta sobre despesas no extrato que podem precisar de notas de entrada
    despesas_do_mes = extrato_do_mes[extrato_do_mes['Valor'] < 0]
    if not despesas_do_mes.empty:
        story.append(Paragraph('<font color="orange">■</font> [ATENÇÃO]: Despesas no extrato que podem precisar de nota de entrada:', normal_style))
        for _, despesa in despesas_do_mes.iterrows():
            descricao = despesa.get('Descricao', 'Sem descrição')
            story.append(Paragraph(f"  - Despesa de {formatar_moeda(despesa['Valor'])} em {despesa['Data'].strftime('%d/%m/%Y')}: {descricao}", normal_style))
    else:
        story.append(Paragraph('<font color="green">■</font> [OK]: Nenhuma despesa sem nota encontrada neste mês.', normal_style))
    story.append(Spacer(1, 0.2*inch))

# 7. --- Resumo Anual ---
story.append(PageBreak())
story.append(Paragraph("Resumo Anual", heading2_style))
story.append(Spacer(1, 0.1*inch))

# Calcula o faturamento anual total e a estimativa de imposto anual
faturamento_anual = notas_df.groupby('Ano')['Valor'].sum()
imposto_anual_total = sum(impostos_mensais.values())

# Adiciona ao PDF o faturamento anual e o alerta de mudança de faixa
if not faturamento_anual.empty:
    story.append(Paragraph("Faturamento Anual:", heading3_style))
    for ano, valor in faturamento_anual.items():
        story.append(Paragraph(f"  - <b>{int(ano)}</b>: {formatar_moeda(valor)}", normal_style))
        story.append(Paragraph(verificar_mudanca_faixa(valor), normal_style))
else:
    story.append(Paragraph('<font color="red">■</font> [ERRO]: Não há dados de faturamento anual.', normal_style))
story.append(Spacer(1, 0.2*inch))
story.append(Paragraph(f"Estimativa de Imposto Anual (Simples Nacional): <b>{formatar_moeda(imposto_anual_total)}</b>", normal_style))
story.append(Spacer(1, 0.2*inch))

# 8. --- Análise de CNPJs em Tempo Real ---
story.append(Paragraph("Análise de CNPJs em Tempo Real", heading2_style))
# Itera sobre cada CNPJ único das notas fiscais
for cnpj in notas_df['CNPJCliente'].unique():
    # Consulta o status legal do CNPJ usando a função auxiliar
    status_legal = verificar_status_cnpj(cnpj)
    # Define o símbolo e a cor com base no status do CNPJ para visualização
    simbolo = {
        'ATIVA': '<font color="green">■</font>',
        'BAIXADA': '<font color="black">■</font>',
        'INAPTA': '<font color="orange">■</font>',
        'TOKEN INVÁLIDO': '<font color="red">■</font>',
        'NÃO ENCONTRADO': '<font color="red">■</font>',
        'ERRO NA CONSULTA': '<font color="red">■</font>',
        'DESCONHECIDO': '<font color="orange">■</font>'
    }.get(status_legal.upper(), '<font color="black">■</font>')
    story.append(Paragraph(f"{simbolo} {cnpj} - SITUAÇÃO: {status_legal}", normal_style))
story.append(Spacer(1, 0.2*inch))

# 9. --- Geração de PDF ---
gerar_pdf(story)
print("Relatório em PDF gerado com sucesso!")
# Inicia o download do arquivo PDF no navegador
files.download("relatorio_fiscal.pdf")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 35.0 MB/s eta 0:00:00
Selecione os arquivos: notas_fiscais e extrato_bancario


Saving extrato_bancario.csv to extrato_bancario.csv
Saving notas_fiscais.csv to notas_fiscais.csv
Colunas do DataFrame de Extrato Bancário após renomear: ['Data', 'Descricao', 'Valor']
Colunas do DataFrame de Notas Fiscais após renomear: ['Data', 'CNPJCliente', 'Valor']
Relatório em PDF gerado com sucesso!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>